# 1 Introduction

## 1.1 Description & Disscusion of the Background

### The city of Chengdu is  the biggest city in south-east China . Chengdu is the capital of Sichuan Province, a mega city, the core city of Chengdu metropolitan area, an important central city in western China approved by the State Council, an important national high-tech industrial base, trade and logistics center and comprehensive transportation hub.By 2020, the city has jurisdiction over 12 districts, 3 counties and 5 county-level cities, with a total area of 14,335 square kilometers.In 2019, the built-up area covered 949.6 square kilometers, with a permanent resident population of 16,581,000 and an urban population of 12,337,900, representing an urbanization rate of 74.41%.

###  Because of its economic importance and large population, chengdu's real estate prices remain high.As a commercial investor, he would like to know where the real estate investment cost will be lower and the commercial competition will be less fierce. In such a region, more commercial income will be obtained with less cost.

### Therefore, in the following part, I will use the method of data science to visualize the housing price of various regions in Chengdu on the map in the form of thermal chart.In addition, I would use the Forsquare Api to get venue information and cluster them on the map.In this way, we will consider the conduct of the business on both of the real estate price and the intense of business competition.

## 1.2 Data Description

### To solve this problem, we need the following data:

### \$ \bigstar\$ I found the average real estate price data of each Borough in Chengdu on the website of www.anjuke.com

### \$ \bigstar\$ I used Forsquare API to get the most common venues of given Borough of Chengdu

### \$ \bigstar\$I used Baidu Map API to obtain the center longitude and latitude of each Borough in Chengdu

### \$ \bigstar\$ I use Aliyun API to get the geojson data of  each Borough in Chengdu